## Random agents with stepped vault APR

### Step 1: Setup experiment parameters

In [ ]:
from elfpy import DEFAULT_LOG_MAXBYTES

log_filename = "../../.logging/random_agent_demo.log" # Output filename for logging
config_file = "../../config/example_config.toml" # Config file to start from (overrides are specified below)

log_level = "DEBUG" # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
max_bytes = DEFAULT_LOG_MAXBYTES # Maximum log file output size, in bytes. More than 100 files will cause overwrites.
pricing_model = "Hyperdrive" # can be yieldspace or hyperdrive

num_agents = 5 # int specifying how many agents you want to simulate
agent_budget = 10_000 # max money an agent can spend

num_trading_days = 3#100  # Number of simulated trading days
blocks_per_day = 20 # Initial vault APR
target_liquidity = 1e7 # target total liquidity of the initial market, before any trades
target_pool_apr = 0.05 # target pool APR of the initial market after the LP
fee_percent = 0.10 # fee percent collected on trades

vault_apr_init = 0.01 # Initial vault APR
vault_apr_jump_size = 0.01 # Size the vault APR can jump
vault_apr_num_jumps = 5 # The average number of jumps to accur in num_trading_days
vault_apr_jump_direction = "random" # The direction of a jump. Can be 'up', 'down', or 'random'.

### Step 2: Setup random agent

In [ ]:
from typing import Generator
import numpy as np
from elfpy.types import MarketActionType, MarketAction, WEI
from elfpy.agent import Agent
from elfpy.markets import Market

class RandomAgent(Agent):
    """
    Agent that randomly opens or closes longs or shorts
    """

    def __init__(self, rng: Generator, wallet_address: int, budget: int = 10_000) -> None:
        """Add custom stuff then call basic policy init"""
        self.rng = rng
        super().__init__(wallet_address, budget)

    def action(self, market: Market) -> list[MarketAction]:
        """Implement a random user strategy

        The agent performs one of four possible trades:
            [OPEN_LONG, OPEN_SHORT, CLOSE_LONG, CLOSE_SHORT]
            with the condition that close actions can only be performed after open actions

        The amount opened and closed is random, within constraints given by agent budget & market reserve levels

        Parameters
        ----------
        market : Market
            the trading market

        Returns
        -------
        action_list : list[MarketAction]
        """
        # User can always open a trade, and can close a trade if one is open
        available_actions = [MarketActionType.OPEN_SHORT, MarketActionType.OPEN_LONG]
        has_opened_short = bool(any((short.balance > 0 for short in self.wallet.shorts.values())))
        if has_opened_short:
            available_actions.append(MarketActionType.CLOSE_SHORT)
        has_opened_long = bool(any((long.balance > 0 for long in self.wallet.longs.values())))
        if has_opened_long:
            available_actions.append(MarketActionType.CLOSE_LONG)
        action_type = self.rng.choice(available_actions, size=1) # choose one random trade type
        # trade amount is also randomly chosen
        match action_type:
            case MarketActionType.OPEN_SHORT:
                random_normal = self.rng.normal(loc=self.budget * 0.1, scale=self.budget * 0.01)
                max_short = self.get_max_short(market)
                if max_short > WEI: # if max_short is greater than the minimum eth amount
                    trade_amount = np.maximum(WEI, np.minimum(max_short, random_normal)) # WEI <= trade_amount <= max_short
                    action_list = [
                        self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                    ]
                else: # no short is possible
                    action_list = []
            case MarketActionType.OPEN_LONG:
                random_normal = self.rng.normal(loc=self.budget * 0.1, scale=self.budget * 0.01)
                max_long = self.get_max_long(market)
                if max_long > WEI: # if max_long is greater than the minimum eth amount
                    trade_amount = np.maximum(WEI, np.minimum(max_long, random_normal))
                    action_list = [
                        self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                    ]
                else:
                    action_list = []
            case MarketActionType.CLOSE_SHORT:
                short_time = self.rng.choice(list(self.wallet.shorts))
                trade_amount = self.rng.uniform(low=WEI, high=self.wallet.shorts[short_time].balance)
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=short_time),
                ]
            case MarketActionType.CLOSE_LONG:
                long_time = self.rng.choice(list(self.wallet.longs))
                trade_amount = self.rng.uniform(low=WEI, high=self.wallet.longs[long_time].balance)
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=long_time),
                ]
        return action_list


def get_example_agents(rng: Generator, budget: float, new_agents: int, existing_agents: int = 0) -> list[Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = RandomAgent(
            rng=rng,
            wallet_address=address,
            budget=budget,
        )
        agent.log_status_report()
        agents += [agent]
    return agents

### Step 3: Define vault apr process

In [ ]:
from elfpy.utils.config import Config

def poisson_vault_apr(
    config: Config, initial_apr: float, jump_size: float, number_of_jumps: int, direction: str
) -> Generator:
    poisson_jumps = config.simulator.rng.poisson(
        lam=number_of_jumps, size=config.simulator.num_trading_days - 1
    ).tolist()
    match direction:
        case "up":
            sign = 1
        case "down":
            sign = -1
        case "random":
            sign = config.simulator.rng.choice([-1, 1], size=1).item() # flip a coin
        case _:
            raise ValueError(f"Direction must be 'up', 'down', or 'random'; not {direction}")
    vault_apr = np.array([initial_apr] * config.simulator.num_trading_days)
    for jump_location in poisson_jumps:
        step = sign * jump_size
        vault_apr[jump_location:] += step
    for apr in vault_apr:
        yield apr

### Step 4: Setup experiment configuration using parameters specified above

In [ ]:
import elfpy.utils.parse_config as config_utils

# parameters set at the top of the notebook
override_dict = {
    "pricing_model_name": pricing_model,
    "num_trading_days": num_trading_days,
    "num_blocks_per_day": blocks_per_day,
    "pricing_model_name": pricing_model,
    "target_liquidity": target_liquidity,
    "target_pool_apr": target_pool_apr,
    "fee_percent": fee_percent,
    "logging_level": log_level,
}
config = config_utils.override_config_variables(config_utils.load_and_parse_config_file(config_file), override_dict)

# override the vault_apr, which is based on some variables set above
override_dict["vault_apr"] = lambda: poisson_vault_apr(
    config=config,
    initial_apr=vault_apr_init,
    jump_size=vault_apr_jump_size,
    number_of_jumps=vault_apr_num_jumps,
    direction=vault_apr_jump_direction,
)
config = config_utils.override_config_variables(config, override_dict)

### Step 5: Run the simulation

In [ ]:
from elfpy.utils import sim_utils
import elfpy.utils.outputs as output_utils

# define root logging parameters
output_utils.setup_logging(
    log_filename=log_filename,
    max_bytes=max_bytes,
    log_level=config_utils.text_to_logging_level(config.simulator.logging_level),
)

# initialize the simulator
random_agents = get_example_agents(rng=config.simulator.rng, budget=agent_budget, new_agents=num_agents, existing_agents=1)
simulator = sim_utils.get_simulator(config, random_agents)

# run the simulation
simulator.run_simulation()

### Step 6: Plot simulation results

In [ ]:
import elfpy.utils.post_processing as post_processing
trades = post_processing.compute_derived_variables(simulator)

In [ ]:
import logging
import matplotlib.pyplot as plt

logging.getLogger().setLevel(logging.WARNING)  # events of this level and above will be tracked

vault_spot_size = 10
spot_colors = ['blue', 'orange']
fig, ax = plt.subplots()
x_data = trades.day
ax.scatter(x_data, trades.vault_apr, label="Vault", s=vault_spot_size, c=spot_colors[0])
prev_apr = trades.loc[trades.run_trade_number==0].pool_apr
prev_time = 0
for day in set(x_data):
    trade_numbers = trades.loc[trades.day==day].run_trade_number
    spot_sizes = np.linspace(0.2, 0.9, len(trade_numbers))
    for trade_idx, trade_number in enumerate(trade_numbers):
        pool_apr = trades.loc[trades.run_trade_number==trade_number].pool_apr
        pool_spot_size = vault_spot_size * 0.5 #spot_sizes[trade_idx]
        time = day + spot_sizes[trade_idx]
        if day == 0 and trade_idx == len(trade_numbers)-1:
            ax.scatter(time, pool_apr, label="Pool", s=pool_spot_size, c=spot_colors[1])
        else:
            ax.scatter(time, pool_apr, s=pool_spot_size, c=spot_colors[1])
        ax.plot([prev_time, time], [prev_apr, pool_apr], color='k', linestyle='-', linewidth=0.1)
        prev_time = time
        prev_apr = pool_apr
ax.set_xlabel("Day")
ax.set_ylabel("APR")
plt.legend()
ax.set_xticks([x for x in range(0, simulator.config.simulator.num_trading_days + 1, 5)])
ax.set_xticklabels([str(x+1) for x in range(0, simulator.config.simulator.num_trading_days + 1, 5)])
ax.set_title("Sawtooth demo")
plt.grid()

In [ ]:
fig = output_utils.plot_wallet_returns(simulator, exclude_first_agent=True, xtick_step=20)

In [ ]:
fig = output_utils.plot_pool_apr(simulator)

In [ ]:
fig = output_utils.plot_market_spot_price(simulator)

In [ ]:
fig = output_utils.plot_market_lp_reserves(simulator)

In [ ]:
fig = output_utils.plot_longs_and_shorts(simulator, xtick_step=20)